In [15]:
print("Hello")

Hello


In [1]:
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
embedding_model=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [4]:
embedding_model.aembed_query("This is my fist query to embed")

<coroutine object GoogleGenerativeAIEmbeddings.aembed_query at 0x000002099DED39A0>

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
import os
pdf_dir=os.path.join(os.getcwd(), "data")

In [9]:
# List all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

# Load all PDFs
documents = []
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_dir, pdf_file)
    loader = PyPDFLoader(file_path)
    documents.extend(loader.load()) 

PdfReadError("Invalid Elementary Object starting with b'P' @18807504: b'7 0 obj<</Universal PDF(The process that creates this PDF constitutes a trade se'")


In [10]:
len(documents)

1217

In [36]:
file_path

'd:\\LLMOPS Industry Projects\\document_portal\\notebook\\data\\sample_data.pdf'

In [42]:
documents = loader.load()

PdfReadError("Invalid Elementary Object starting with b'P' @18807504: b'7 0 obj<</Universal PDF(The process that creates this PDF constitutes a trade se'")


In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap = 150,
    length_function = len
)

In [12]:
docs= text_splitter.split_documents(documents)

In [13]:
len(docs)

10775

In [14]:
docs[100].metadata

{'producer': 'pdfTeX-1.40.3',
 'creator': 'Elsevier',
 'creationdate': '2015-06-13T13:55:15+05:30',
 'author': 'Cheikh Kacfah Emani',
 'crossmarkdomains[1]': 'sciencedirect.com',
 'crossmarkdomains[2]': 'elsevier.com',
 'crossmarkdomainexclusive': 'true',
 'crossmarkmajorversiondate': '2010-04-23',
 'elsevierwebpdfspecifications': '6.4',
 'moddate': '2015-06-13T13:55:27+05:30',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-1.40.10-2.2 (TeX Live 2009/Debian) kpathsea version 5.0.0',
 'subject': 'Computer Science Review, Corrected proof. doi:10.1016/j.cosrev.2015.05.002',
 'title': 'Understandable Big Data: A survey',
 'trapped': '/False',
 'doi': '10.1016/j.cosrev.2015.05.002',
 'robots': 'noindex',
 'source': 'd:\\LLMOPS Industry Projects\\document_portal\\notebook\\data\\2. Understanding Big ata.pdf',
 'total_pages': 12,
 'page': 1,
 'page_label': '2'}

In [15]:
from langchain.vectorstores import FAISS

In [17]:
docs[105].page_content

'size is beyond the ability of typical database software tools\nto capture, store, manage, and analyze”. Likewise, Davis and\nPatterson [1, page 4] say “Big data is data too big to be handled\nand analyzed by traditional database protocols such as SQL”;\nand the same opinion is shared by [11,3,4], etc. Both groups of\nauthors previously mentioned go beyond the only size aspects\nof data when defining Big Data! Edd Dumbill in [12, page 3]\nexplicitly conveys the multi-dimensionality of Big Data when'

In [ ]:
embedding_model.embed_documents((docs[105].page_content))

In [1]:
vectorstore = FAISS.from_documents(docs, embedding_model)

NameError: name 'FAISS' is not defined

In [26]:
docs_retreived = vectorstore.similarity_search('what ajk policy lakes')

In [27]:
for doc in docs_retreived:
    print(doc)

page_content='The Kashmir Digital Policy 2024 -2030 outlines a comprehensive framework aimed at enhancing access, 
connectivity, and digital infrastructure while also promoting digital governance, legislation, human resource 
development, and private sector innovation. 
A critical analysis of this policy reveals several strengths and potential weaknesses: 
Strengths: 
1. Holistic Approach : The policy covers multiple core areas, indicating a well -rounded strategy that' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-13T05:28:42+05:00', 'author': 'arshad abbas', 'moddate': '2024-09-13T05:28:42+05:00', 'source': 'd:\\LLMOPS Industry Projects\\document_portal\\notebook\\data\\The Kashmir Digital Policy 2024.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
page_content='implemented in educational institutions to develop skilled AI practitioners and data scientists. 
In conclusion, the Kashmir Digital 

In [76]:
retreiver = vectorstore.as_retriever()

In [ ]:
retreiver.invoke('search agents provides several opportunities for learning')

In [63]:
from langchain.prompts import PromptTemplate

prompt_template = """
Check the context and answer the question if you dont find any answer then return  o answer found

context: {context}

question {question}

Answer

"""

In [64]:
prompt = PromptTemplate ( template=prompt_template,
                         input_variables= ['context','question'])

In [65]:
from langchain_core.output_parsers import StrOutputParser

In [67]:
output_parser = StrOutputParser()

In [68]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [69]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [73]:
from langchain.schema.runnable import RunnablePassthrough

In [74]:
rag_chain = (
    {"context": retreiver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [78]:
rag_chain.invoke('what is Metropolis algorithm')

'<think>\nOkay, so I need to figure out what the Metropolis algorithm is based on the given context. Let me read through the context carefully.\n\nFirst, I see that the context mentions "MCMC algorithms" which stand for Markov Chain Monte Carlo. It talks about generating samples by making random changes to the preceding sample, which is a key part of MCMC methods.\n\nThen, there\'s a section about the Metropolis–Hastings algorithm. It says that this algorithm operates in two stages. The first stage is sampling a new state x\' from a proposal distribution q(x\'|x) given the current state x. The second stage is about accepting or rejecting this new state with a certain probability α(x\'|x). The acceptance probability is calculated as the minimum of 1 and the ratio of π(x\')q(x|x\') to π(x)q(x\'|x). If the proposal is rejected, the state stays at x.\n\nSo, putting this together, the Metropolis algorithm, which is a specific case of Metropolis–Hastings, uses a symmetric proposal distributi